### ETL Pipeline for Pre-Processing the Files

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data would look like in the <font color=blue>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [39]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace 

In [40]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                   ) 
except Exception as e:
    print(e)

#### Setting KEYSPACE to the keyspace specified above

In [41]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Modelling the database tables based on the user queries

#### Query 1  
Provide the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

The above query is filtering the data based on sessionId and itemInSession. The data can be distributed using the sessionId and sorted using the itemInSession column. The Primary Key for this table can be defined using sessionId as partition key and itemInSession as clustering column to uniquely identify each row

In [42]:
query = "CREATE TABLE IF NOT EXISTS session_details "
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)         

In [43]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_details (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [44]:
query = "select artist, song, length from session_details WHERE sessionId = 338 and itemInSession  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### Query 2
Provide the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The above query is filtering the data based on userId and sessionId and needs the songs to be sorted based on the song column. The Primary Key for this table can be defined using a composite key of userId and sessionId for partitioning and itemInSession as clustering column

In [45]:
query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
    
except Exception as e:
    print(e)         

In [46]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [47]:
query = "select artist, song, firstName, lastName from user_songs WHERE userId = 10 and sessionId  = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3 
Provide the user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The above query is filtering the data based on the song column. But the song column alone cannot uniquely identify each record. The Primary Key for this table can be defined using song as the partition key and userId as clustering column to uniquely identify each row

In [48]:
query = "CREATE TABLE IF NOT EXISTS users_for_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
    
except Exception as e:
    print(e)                      

In [49]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_for_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [50]:
query = "select firstName, lastName from users_for_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [51]:
try:
    query = "DROP TABLE session_details"
    session.execute(query)
    query = "DROP TABLE user_songs"
    session.execute(query)
    query = "DROP TABLE users_for_song"
    session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()